In [21]:
FONT_LINKS = """
https://web.archive.org/web/20130404225932/http://www.iol.ie/~sob/gadelica.zip
http://web.archive.org/web/20120830224624/http://www.gaelchlo.com/bunardgc.zip
http://web.archive.org/web/20220322030900/https://www.gaelchlo.com/bunargc.zip
http://web.archive.org/web/20120830224559/http://www.gaelchlo.com/bunartgc.zip
http://web.archive.org/web/20191216105737/http://www.gaelchlo.com/bundgc.zip
http://web.archive.org/web/20120830224343/http://www.gaelchlo.com/bunnod.zip
http://web.archive.org/web/20191218121436/http://www.gaelchlo.com/buntgc.zip
http://web.archive.org/web/20160408212120/http://gaelchlo.com/seandgc.zip
http://web.archive.org/web/20160408212013/http://gaelchlo.com/seantgc.zip
http://web.archive.org/web/20220322030900/https://www.gaelchlo.com/bunargc.zip
http://web.archive.org/web/20240119141814/https://www.gaelchlo.com/bungc.zip
http://web.archive.org/web/20160408212120/http://gaelchlo.com/seandgc.zip
http://web.archive.org/web/20240119143113/https://www.gaelchlo.com/seangc.zip
http://web.archive.org/web/20160408212013/http://gaelchlo.com/seantgc.zip
http://web.archive.org/web/20240619180031/http://www.gaelchlo.com/urgc.zip
http://web.archive.org/web/20240119141944/https://www.gaelchlo.com/aongc.zip
http://web.archive.org/web/20240619182847/https://www.gaelchlo.com/ardeorgc.zip
http://web.archive.org/web/20240119142032/https://www.gaelchlo.com/ardgc.zip
http://web.archive.org/web/20240119142040/https://www.gaelchlo.com/barrgc.zip
http://web.archive.org/web/20240119142035/https://www.gaelchlo.com/casgc.zip
http://web.archive.org/web/20240619182918/https://www.gaelchlo.com/corrgc.zip
http://web.archive.org/web/20240119142042/https://www.gaelchlo.com/deasgc.zip
http://web.archive.org/web/20240119142045/https://www.gaelchlo.com/dluthgc.zip
http://web.archive.org/web/20240119142026/https://www.gaelchlo.com/dubhgc.zip
http://web.archive.org/web/20240119142025/https://www.gaelchlo.com/fiorgc.zip
http://web.archive.org/web/20240119142051/https://www.gaelchlo.com/geargc.zip
http://web.archive.org/web/20240119142055/https://www.gaelchlo.com/glangc.zip
http://web.archive.org/web/20240119143123/https://www.gaelchlo.com/langc.zip
http://web.archive.org/web/20240619183031/https://www.gaelchlo.com/lomgc.zip
http://web.archive.org/web/20240619183107/https://www.gaelchlo.com/meargc.zip
http://web.archive.org/web/20240619183125/https://www.gaelchlo.com/mingc.zip
"""

In [22]:
FONTS = [x for x in FONT_LINKS.split("\n") if x != ""]

In [ ]:
for font in FONTS:
  !wget {font}
  !unzip {font.split("/")[-1]}

In [11]:
!mkdir fonts
!mv *.ttf fonts

In [ ]:
%%capture
!apt install python3-fontforge

In [ ]:
%%capture
!apt install lynx

In [40]:
!lynx -dump https://www.gaelchlo.com/clonna1.html|grep html|grep '[12][0-9]\.'|awk '{print $2}'|while read i;do lynx -dump $i|grep zip;done|grep https|awk '{print $2}'|sort|uniq > fontlist

In [ ]:
!wget -i fontlist

In [ ]:
%%capture
!for i in *zip;do unzip -o $i;done

In [52]:
!cat fontlist|sort|uniq|grep -v 'w.zip$'|awk '{print "http://web.archive.org/web/" $0}' > iafonts
#!wget -i iafonts -o iafonts.log

In [70]:
from pathlib import Path
import fontforge

for otf in Path(".").glob("**/*.otf"):
    font = fontforge.open(str(otf))
    outname = "fonts/" + otf.stem + ".ttf"
    font.save(outname)

In [50]:
!grep '^-' iafonts.log|grep 'web/2'|awk '{print $NF}' > iafonts-arc

In [51]:
!wc -l iafonts*

   38 iafonts
   33 iafonts-arc
  545 iafonts.log
  616 total


In [59]:
!rm redo
!cat iafonts|sed -e 's#http://web.archive.org/web/##'|while read i;do grep $i iafonts-arc || echo http://web.archive.org/web/$i >> redo;done

http://web.archive.org/web/20240119141944/https://www.gaelchlo.com/aongc.zip
http://web.archive.org/web/20240619182847/https://www.gaelchlo.com/ardeorgc.zip
http://web.archive.org/web/20240119142032/https://www.gaelchlo.com/ardgc.zip
http://web.archive.org/web/20240119142040/https://www.gaelchlo.com/barrgc.zip
http://web.archive.org/web/20240119141814/https://www.gaelchlo.com/bungc.zip
http://web.archive.org/web/20240119142035/https://www.gaelchlo.com/casgc.zip
http://web.archive.org/web/20240619182918/https://www.gaelchlo.com/corrgc.zip
http://web.archive.org/web/20240119142042/https://www.gaelchlo.com/deasgc.zip
http://web.archive.org/web/20240119142045/https://www.gaelchlo.com/dluthgc.zip
http://web.archive.org/web/20240119142026/https://www.gaelchlo.com/dubhgc.zip
http://web.archive.org/web/20240119142025/https://www.gaelchlo.com/fiorgc.zip
http://web.archive.org/web/20240119142051/https://www.gaelchlo.com/geargc.zip
http://web.archive.org/web/20240119142055/https://www.gaelchlo.co

In [61]:
!wget -i redo

--2024-06-19 18:45:07--  http://web.archive.org/web/https://www.gaelchlo.com/miongc.zip
Resolving web.archive.org (web.archive.org)... 207.241.237.3
Connecting to web.archive.org (web.archive.org)|207.241.237.3|:80... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: http://web.archive.org/save/_embed/https://www.gaelchlo.com/miongc.zip [following]
--2024-06-19 18:45:07--  http://web.archive.org/save/_embed/https://www.gaelchlo.com/miongc.zip
Reusing existing connection to web.archive.org:80.
HTTP request sent, awaiting response... 302 FOUND
Location: http://web.archive.org/web/20240619184509/https://www.gaelchlo.com/miongc.zip [following]
--2024-06-19 18:45:14--  http://web.archive.org/web/20240619184509/https://www.gaelchlo.com/miongc.zip
Reusing existing connection to web.archive.org:80.
HTTP request sent, awaiting response... 302 FOUND
Location: http://web.archive.org/save/_embed/https://www.gaelchlo.com/miongc.zip [following]
--2024-06-19 18:45:14--  http://we

In [ ]:
!wget https://web.archive.org/web/20240619184509/https://www.gaelchlo.com/miongc.zip
!wget http://web.archive.org/web/20240619184517/https://www.gaelchlo.com/morgc.zip
!wget http://web.archive.org/web/https://inkwell.ie/font-packs/Macalla/Macalla-Regular-ttf.zip

In [68]:
%%capture
!pip install trdg

In [69]:
!trdg -h

2024-06-19 19:08:29.921778: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-19 19:08:29.921953: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-19 19:08:30.133667: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-19 19:08:30.571011: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-19 19:08:33.968024: W tensorflow/compiler/tf2